In [1]:
import requests
import os
import json

## Parte 4
### Popular o seu banco de dados

Apos criar o DB acima, devemos saber como trabalhar com os dados dele.  A forma utilizada para guardar esses dados não relacionais são os documentos, que são escritos em formato json. Para organizar os dados, cada documneto tem um id próprio, que pode ser qualquer coisa, mas é recomentdado utilizar o UUID, que inclusive pode ser gerado pela API do próprio couchdb.

Antes disso porem, devemos colocar as variáveis abaixo, para podermos acessar corretamente os DBs

In [65]:
username = 'francisco'
senha = 'Maroav01'
ip = '127.0.0.1'
porta = '5984'
nome_db = "bas"

Para criar um novo documento para popular o banco de dados, vamos primeiro gerar um id para o documento. Faremos isso enviando uma chamada 'GET' no formato a seguir:       
'http://< endereco couchdb>:< porta couchdb>/_uuids'
    
 

In [4]:
# Endereço para gerar o ID 
res = requests.get('http://%s:%s/_uuids' % (ip,porta))
res.json()

# Salvando o ID gerado
uuid = res.json()['uuids'][0]
uuid

'ec61b8937307fa7e895590aae5000c10'

Agora que geramos e salvamos o ID para nosso novo documento, vamos gerar o documento e adicionar ao banco de dados. Para isso vamos colocar em json os dados. Vamos salvar nomes de pessoas e que instrumentos elas tocam no formato a seguir:

{
"Key1" : "Valor1",
"Key2" : "Valor2",
"Key3" : "Valor3",
...
}

E enviamos esses dados no *body* da requisição, com o formato a seguir usando **' PUT '** :

'http://< usuario >:< senha >@< endereco couchdb>:< porta couchdb>/< nome DB>/< id do documento >'


In [8]:
# Vamos colocar no banco os dados
dados_a_serem_salvos = '{"Nome":"Carlos","Instrumento":"Guitarra"}'

res = requests.put('http://%s:%s@%s:%s/%s/%s' % (username,senha,ip,porta,nome_db,uuid),data=dados_a_serem_salvos)
print(res.json())

{'ok': True, 'id': 'ec61b8937307fa7e895590aae5000c10', 'rev': '1-4a3cb778198a80655cfd3f65a9ac53c4'}


Vamos colocar mais alguns dados, para popular o banco

In [9]:
dados_a_serem_salvos = ['{"Nome":"Pedro","Instrumento":"Guitarra"}','{"Nome":"Magda","Instrumento":"Piano"}','{"Nome":"Jubileu","Instrumento":"Saxofone"}']

for i in dados_a_serem_salvos:
    
    res = requests.get('http://%s:%s/_uuids' % (ip,porta))
    # Salvando o ID gerado
    uuid = res.json()['uuids'][0]
    # Salvando dados
    res = requests.put('http://%s:%s@%s:%s/bas/%s' % (username,senha,ip,porta,uuid),data=i)
    print(res.json())

{'ok': True, 'id': 'ec61b8937307fa7e895590aae50013bc', 'rev': '1-cfb138b570e0c7c98ff31735d996284e'}
{'ok': True, 'id': 'ec61b8937307fa7e895590aae5001aba', 'rev': '1-cfbb34f9fd12236fd4910489fa44e52e'}
{'ok': True, 'id': 'ec61b8937307fa7e895590aae5002164', 'rev': '1-dc95a46d02f715a00dcba69c0ec80342'}


Para ver os ids de todos os documentos de um DB podemos utilizar a requisição no formato

'http://< endereco couchdb>:< porta couchdb >/< nome DB >/_all_docs'


In [10]:
res = requests.get('http://%s:%s/bas/_all_docs' % (ip,porta))
res.json()

{'total_rows': 4,
 'offset': 0,
 'rows': [{'id': 'ec61b8937307fa7e895590aae5000c10',
   'key': 'ec61b8937307fa7e895590aae5000c10',
   'value': {'rev': '1-4a3cb778198a80655cfd3f65a9ac53c4'}},
  {'id': 'ec61b8937307fa7e895590aae50013bc',
   'key': 'ec61b8937307fa7e895590aae50013bc',
   'value': {'rev': '1-cfb138b570e0c7c98ff31735d996284e'}},
  {'id': 'ec61b8937307fa7e895590aae5001aba',
   'key': 'ec61b8937307fa7e895590aae5001aba',
   'value': {'rev': '1-cfbb34f9fd12236fd4910489fa44e52e'}},
  {'id': 'ec61b8937307fa7e895590aae5002164',
   'key': 'ec61b8937307fa7e895590aae5002164',
   'value': {'rev': '1-dc95a46d02f715a00dcba69c0ec80342'}}]}

Vamos fazer uma consulta em um documento específico, para isso basta fazer uma requisição GET no formato : 

'http://< endereco couchdb>:< porta couchdb >/< nome DB >/< id do documento >'



In [11]:
res = requests.get('http://%s:%s/bas/%s' % (ip,porta,uuid))
res.json()

{'_id': 'ec61b8937307fa7e895590aae5002164',
 '_rev': '1-dc95a46d02f715a00dcba69c0ec80342',
 'Nome': 'Jubileu',
 'Instrumento': 'Saxofone'}

Para editar um documento, devemos fazer uma revisão do mesmo. Ao fazer uma revisão, carregamos o arquivo insteiro e então alteramos a parte nescessária, e então identificamos tambem o id de revisão, que é um valor chamado '*rev*', o qual deve ser enviado para evitar que no meio tempo que o doc foi carregado e alterado, outra pessoa tenha alterado ja antes e estejamos sobrescrevendo algo importante.

O valor '*rev*' vem junto do documento quando carregado, e ele é alterado a cada revisão feita.
Vamos revisar o documento que puxamos no exemplo acima.

In [12]:
rev = res.json()['_rev']

nova_alteracao = '{"Nome":"Jose","Instrumento":"Saxofone", "_rev":"%s"}' % (rev) 


res1 = requests.put('http://%s:%s@%s:%s/bas/%s' % (username,senha,ip,porta,uuid),data=nova_alteracao)

res1.json()

{'ok': True,
 'id': 'ec61b8937307fa7e895590aae5002164',
 'rev': '2-116e9ac592a7df58a5409b2f85d8506e'}

Uma vez que a revisão é feita o sistema gera um novo valor para rev e o retorna para o usuário. Agora podemos tambem apagar lógicamente um documento. Nese caso o formato do requisição "**DELETE**" é:

'http://< usuario >:< senha >@< endereco couchdb>:< porta couchdb >/< nome DB >/< id do documento >?rev=< valor rev do documento >'

Vamos testar isso apagando o arquivo que acabamos de alterar. 

In [13]:
rev = res1.json()['rev']
json_delete = '{"_rev":"%s"}' % (rev)
res1 = requests.delete('http://%s:%s@%s:%s/bas/%s?rev=%s' % (username,senha,ip,porta,uuid,rev))
res1.json()

{'ok': True,
 'id': 'ec61b8937307fa7e895590aae5002164',
 'rev': '3-b6553ccb034664771b3288ca5c48dee2'}

Assim podemos ver que se fizermos a requisição para ver o documento o resultado retorna que os dados não foram encontrados pois foram apagados.

In [14]:
res = requests.get('http://%s:%s/bas/%s' % (ip,porta,uuid))
res.json()

{'error': 'not_found', 'reason': 'deleted'}

## Parte 5
### Map Reduce

O mapreduce são mapas e funções reduces predefinidas e programadas em javascript, que serverm para selecionar os dados requeridos dos documentos sem se preocupar com a formatação de cada documento, facilitando a busca quando o DB for bastante variado. 

Para criar uma view, que é como se chamam essas funções do couchdb, é preciso criar um documento que irá conter todos os views criados. O formato do requerimento **' PUT '** é:

'http://< usuario >:< senha >@< endereco couchdb>:< porta couchdb >/< nome DB >/_design/< nome do documento de views >'

E a função é enviada em um json com o seguinte formato:

{"views": {"< Nome da View >": {"map": "< Função javascript do view >"}},"language": "javascript"}

No caso atual, vamos criar uma view para puxar os nomes de todos os docs.

*function(doc) {emit(doc.Nome);}*

A função em javascript recebe em *doc* todos os dados de cada documento, e a função *emit* é utilizada para enviar os dados quando ocorre a requisição 



In [15]:

jj = {"views": {"new-view": {"map": "function(doc) {\n  var key;\n  var instrumentos = {};\n  \n    emit(0,doc.Nome)\n  \n \n}"}},"language": "javascript"}


res = requests.put('http://%s:%s@%s:%s/bas/_design/doc_view' % (username,senha,ip,porta),data=json.dumps(jj))
print(res.json())

{'ok': True, 'id': '_design/doc_view', 'rev': '1-5d2b78f81a263e4b9b20c3a8794d4936'}


Para ver o resultado do view, é so fazer uma requisição **' GET '** com o formato a seguir:

'http://< endereco couchdb>:< porta couchdb >/< nome DB >/_design/< nome do documento de views >/view/< Nome do view >'

In [16]:
res = requests.get('http://%s:%s@%s:%s/bas/_design/doc_view/_view/new-view' % (username,senha,ip,porta))
res.json()

{'total_rows': 3,
 'offset': 0,
 'rows': [{'id': 'ec61b8937307fa7e895590aae5000c10',
   'key': 0,
   'value': 'Carlos'},
  {'id': 'ec61b8937307fa7e895590aae50013bc', 'key': 0, 'value': 'Pedro'},
  {'id': 'ec61b8937307fa7e895590aae5001aba', 'key': 0, 'value': 'Magda'}]}

## Parte 6
### Replicação local

Replicação é criar uma cópia do DB que pode ser sincronizado constantemente. Inicialmente faremos uma replicação local para demonstração. O formato da requisição é:

'http://< usuario >:< senha >@< endereco couchdb>:< porta couchdb >/_replicate'

E o formato dos dados a serem enviados juntos é:


{"source": "< endereco do DB a ser replicado >", 

"target": "< endereço a receber os dados >", 

"create_target": < booleano para definir se precisa criar o DB target >, 

"continuous": < booleano para definir se a replicação manterá os DBs sincronizados >} '




In [51]:
json_data = json.dumps({"source": "%s:%s@%s:%s/bas" % (username,senha,ip,porta), "target": "http://%s:%s@%s:%s/bas2" % (username,senha,ip,porta), "create_target": True, "continuous": True})
headers = {'content-type': 'application/json'}
res = requests.post("http://%s:%s@%s:%s/_replicate" % (username,senha,ip,porta), data = json_data, headers=headers)
res.json()

{'ok': True,
 '_local_id': '4bc9402ccfa7c29936498ecfd765fd5f+continuous+create_target'}

No caso da replicação acima, a replicação acima so funciona em uma diração, fazendo com que o novo DB que criamos seja somente para backup. Para fazer com que os DBs estejam totalmente sincronizados seria preciso criar mais uma ação de replicação para fazer o caminho inverso. 

In [62]:
json_data = json.dumps({"source": "%s:%s@%s:%s/bas2" % (username,senha,ip,porta), "target": "http://%s:%s@%s:%s/bas" % (username,senha,ip,porta), "create_target": False, "continuous": True})
headers = {'content-type': 'application/json'}
res = requests.post("http://%s:%s@%s:%s/_replicate" % (username,senha,ip,porta),data = json_data,headers=headers)
res.json()

{'ok': True, '_local_id': 'b4e46774149d961407f40429b7d9083e+continuous'}

Agora para testar a replicação, vamos colocar um documento novo no DB original, e então verificar a replicação no outro.

In [63]:
i = '{"Nome":"Jake","Instrumento":"Gaita"}'

res = requests.get('http://%s:%s/_uuids' % (ip,porta))
# Salvando o ID gerado
uuid = res.json()['uuids'][0]
# Salvando dados
res = requests.put('http://%s:%s@%s:%s/bas/%s' % (username,senha,ip,porta,uuid),data=i)
print(res.json())

{'ok': True, 'id': 'ec61b8937307fa7e895590aae5005258', 'rev': '1-21451721842b97d2a75bda6c91a14209'}


In [64]:
res = requests.get('http://%s:%s@%s:%s/bas/_design/doc_view/_view/new-view' % (username,senha,ip,porta))
res.json()

{'total_rows': 7,
 'offset': 0,
 'rows': [{'id': 'ec61b8937307fa7e895590aae5000c10',
   'key': 0,
   'value': 'Carlos'},
  {'id': 'ec61b8937307fa7e895590aae50013bc', 'key': 0, 'value': 'Pedro'},
  {'id': 'ec61b8937307fa7e895590aae5001aba', 'key': 0, 'value': 'Magda'},
  {'id': 'ec61b8937307fa7e895590aae5002620', 'key': 0, 'value': 'Jske'},
  {'id': 'ec61b8937307fa7e895590aae5003282', 'key': 0, 'value': 'Jake'},
  {'id': 'ec61b8937307fa7e895590aae5003f8f', 'key': 0, 'value': 'Jake'},
  {'id': 'ec61b8937307fa7e895590aae5005258', 'key': 0, 'value': 'Jake'}]}

In [53]:
res = requests.get('http://%s:%s@%s:%s/bas2/_design/doc_view/_view/new-view' % (username,senha,ip,porta))
res.json()

{'total_rows': 6,
 'offset': 0,
 'rows': [{'id': 'ec61b8937307fa7e895590aae5000c10',
   'key': 0,
   'value': 'Carlos'},
  {'id': 'ec61b8937307fa7e895590aae50013bc', 'key': 0, 'value': 'Pedro'},
  {'id': 'ec61b8937307fa7e895590aae5001aba', 'key': 0, 'value': 'Magda'},
  {'id': 'ec61b8937307fa7e895590aae5002620', 'key': 0, 'value': 'Jske'},
  {'id': 'ec61b8937307fa7e895590aae5003282', 'key': 0, 'value': 'Jake'},
  {'id': 'ec61b8937307fa7e895590aae5003f8f', 'key': 0, 'value': 'Jake'}]}